# CNN-RNN

In [1]:
import tensorflow.keras as keras

In [2]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import os as os
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from time import time
from tensorflow.keras.models import load_model

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [4]:
keras.__version__

'2.2.4-tf'

# 12-FOLD

In [5]:
subjects = ['01', '02', '03', '05', '08', '09', '10', '11', '13', '14', '16', '17']
global activities
activities = [9,10,11,12,13,19,20,21,24,25,31]

In [6]:
def write(name, ts_seg, y_true, y_pred):
    fo = open(name, "w")
    head = "QUAT,w,x,y,z,timestamp,y_true,y_pred\n"
    fo.seek(0,2)
    fo.write(head)
    fo.close()
    
    fo = open(name, "a")
    salida = np.reshape(ts_seg, (ts_seg.shape[0]*ts_seg.shape[2], ts_seg.shape[3]))
    
    k = 0
    for i in range(salida.shape[0]):
        if(i%128==0):
            if(i!=0):
                k+=1
        print('%d'%(i), end='\r')
        fo.write("quat0,"+str(salida[i][0])+","+str(salida[i][1])+","+str(salida[i][2])+","+str(salida[i][3])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        fo.write("quat1,"+str(salida[i][4])+","+str(salida[i][5])+","+str(salida[i][6])+","+str(salida[i][7])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        fo.write("quat2,"+str(salida[i][8])+","+str(salida[i][9])+","+str(salida[i][10])+","+str(salida[i][11])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        fo.write("quat3,"+str(salida[i][12])+","+str(salida[i][13])+","+str(salida[i][14])+","+str(salida[i][15])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        fo.write("quat4,"+str(salida[i][16])+","+str(salida[i][17])+","+str(salida[i][18])+","+str(salida[i][19])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        if(salida.shape[1]==36):
            fo.write("quat5,"+str(salida[i][20])+","+str(salida[i][21])+","+str(salida[i][22])+","+str(salida[i][23])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat6,"+str(salida[i][24])+","+str(salida[i][25])+","+str(salida[i][26])+","+str(salida[i][27])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat7,"+str(salida[i][28])+","+str(salida[i][29])+","+str(salida[i][30])+","+str(salida[i][31])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat8,"+str(salida[i][32])+","+str(salida[i][33])+","+str(salida[i][34])+","+str(salida[i][35])+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
        else:
            fo.write("quat5,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat6,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat7,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
            fo.write("quat8,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+str(activities[int(y_true[k])])+","+str(activities[int(y_pred[k])])+'\n')
    fo.close()

In [7]:
# Ajustado a los datos
n_time_steps = 128
n_classes = 11 # Nº de clases (el 0 está eliminado)
n_channels = 1 # Nº de canales
n_columns = 40

# Podríamos variarlo
batch_size = 128 # Tamaño del batch
learning_rate = 1e-3 # Learning rate (por defNORMecto es 0.001)
epochs = 100 # Épocas

In [8]:
for i in range(12):
    print('##################################################################################################################################')
    print('#####################################################  K-FOLD %d  #################################################################'%(i+1))
    
    subject_test = subjects[i]
    print('Test subject: ' + str(subject_test))
    
    ##  GENERACIÓN DATASET
    ts_seg_norm = np.load('./defNORM_raworientation/subject_video_'+subject_test+'_seg.npy')
    ts_lab_norm = np.load('./defNORM_raworientation/subject_video_'+subject_test+'_lab.npy')
    
    ts_seg = np.load('./def_raworientation_ALL_SENSORS/subject_video_'+subject_test+'_seg.npy')
    
    limit = 40
    ts_seg_norm = ts_seg_norm[:,:,:,:limit]
    ts_seg = ts_seg[:,:,:,:limit]
    ####################################################################################################################
    ## SHUFFLE DE DATOS
    np.random.seed(235)
    ts_seg_norm = np.reshape(ts_seg_norm[np.random.shuffle(np.arange(0,ts_seg_norm.shape[0]))], (98,1,128,limit))
    ts_lab_norm = np.reshape(ts_lab_norm[np.random.shuffle(np.arange(0,ts_seg_norm.shape[0]))], (98,11))
    ts_seg = np.reshape(ts_seg[np.random.shuffle(np.arange(0,ts_seg.shape[0]))], (98,1,128,36))
    
    print('Test dataset: ')
    print(ts_seg_norm.shape, ts_lab_norm.shape)
    ####################################################################################################################
    ## RED
    model = load_model('./01AUGNORMFFT_SP_KFOLD_models/CNN_'+subject_test+'.h5')
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate),metrics=['categorical_accuracy'])
    ####################################################################################################################
    ## TEST
    predictions = model.predict(ts_seg_norm, batch_size = batch_size, verbose = 0)
    
    y_pred = np.argmax(predictions,axis=-1)
    y_pred = np.expand_dims(y_pred,axis=-1)
    
    y_true = np.argmax(ts_lab_norm,axis=-1)
    y_true = np.expand_dims(y_true,axis=-1)
    
    y_true_flat = np.ndarray.flatten(y_true)
    y_pred_flat = np.ndarray.flatten(y_pred)
    
    name = './01AUGNORMFFT_SP_KFOLD_models/RESULT_CNN_'+subject_test+'.csv'
    write(name, ts_seg, y_true, y_pred)
    
    del subject_test, ts_seg_norm, ts_lab_norm, model, y_pred, y_true, predictions, y_true_flat, y_pred_flat, ts_seg
    

##################################################################################################################################
#####################################################  K-FOLD 1  #################################################################
Test subject: 01
Test dataset: 
(98, 1, 128, 40) (98, 11)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
##################################################################################################################################
#####################################################  K-FOLD 2  ############